In [1]:
import pdfplumber
import json
import pandas as pd

In [2]:
# 股票代码检查
import tushare as ts
ts_token = '53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43'

ts.set_token(ts_token)
pro = ts.pro_api()
stock_codes = list(pro.stock_basic(exchange='', list_status='L', fields='ts_code')['ts_code'])

# 以下操作便于之后为股票代码添加后缀
stock_code_dict = {}
for code in stock_codes:
    stock_code_dict[code[:6]] = code

In [3]:
pdf_path = "行业分类.pdf"
pdf =  pdfplumber.open(pdf_path)

In [4]:
pages = pdf.pages
industries = {}
for i in range(len(pages)):
    page = pages[i]
    table = page.extract_table()
    for j in range(len(table)):
        if j == 0:
            continue
        if table[j][0] != None:
            industry = table[j][0].replace(" ", "").replace("\n","")         
            if industry not in industries.keys():
                industries[industry] = []
        stock = table[j][3]
        if stock not in industries[industry] and stock in stock_code_dict.keys():
            industries[industry].append(stock_code_dict[stock])

In [5]:
df = pd.DataFrame.from_dict(industries, orient='index').T
df.to_csv("industry.csv", index = True, sep=',')

In [6]:
# 全部行业
list(industries.keys())

['农、林、牧、渔业(A)',
 '采矿业(B)',
 '制造业(C)',
 '电力、热力、燃气及水生产和供应业(D)',
 '建筑业(E)',
 '批发和零售业(F)',
 '交通运输、仓储和邮政业(G)',
 '住宿和餐饮业(H)',
 '信息传输、软件和信息技术服务业(I)',
 '金融业(J)',
 '房地产业(K)',
 '租赁和商务服务业(L)',
 '科学研究和技术服务业(M)',
 '水利、环境和公共设施管理业(N)',
 '居民服务、修理和其他服务业(O)',
 '教育(P)',
 '卫生和社会工作(Q)',
 '文化、体育和娱乐业(R)',
 '综合(S)']

In [7]:
# 全部股票列表
stock_list = []
for industry in industries:
    stock_list.extend(industries[industry])

stock_codes_file = open('stock.txt', 'w')
for stock_code in stock_list:
	stock_codes_file.write(stock_code)
	stock_codes_file.write('\n')
stock_codes_file.close()

In [8]:
df = pd.DataFrame(stock_list, orient='index').T
df.to_csv("stock.csv", index = True, sep=',')

TypeError: __init__() got an unexpected keyword argument 'orient'